# Migrating workflows from KNIX to AWS Stepfunctions

KNIX is compatible with AWS Lambda and Step Functions with expanded support for sophisticated parallel executions.

## Goal for this Notebook:
Show a simple example of migrating a sample workflow in Python, using SDKs provided for both AWS and KNIX. This is aimed for those looking to get into the field or those who are already in the field and looking to see an example how to move existing workflows between AWS and KNIX.

### This Notebook will show basic examples of:
* Importing SDKs
* Generating and using SDK objects
* converting KNIX names to valid ARNs
* Importing and exporting worflow and function definitions

### Required Libraries:
* [json] (http://www.json.org/)
* [zipfile] (https://docs.python.org/3/library/zipfile)

### Things to remember:
* In Step-Functions workflow descriptions, 'Resource' ARN needs to be changed into real Lambda ARNs. This can be achieved by prepending a fixed, user-specific prefix of the form of "arn:aws:lambda:eu-central-1:123456789012:function:" . Note this step does not require any change in user code.

* In Lambda,  function handler needs to be configured when creating the function. In KNIX the function handler name must always be called "handle". Note this step does not require changing user code.

* KNIX users should put the libraries that they would like to be part of LD_LIBRARY_PATH in a ./lib/ folder, which is inside their deployment zip and sits parallel to their fuction code (referring to the .py file that has the 'handle' method)

* In KNIX, if user's deployment zip contains ELF executable binaries that can be invoked from the python code (using the subprocess module), then these binaries should be invoked using their complete path, and not via symbolic links to them.

* User code in Lambda is only allowed to create files in /tmp, whereas, in KNIX the entire filesystem is writable.


## Now let's start to migrate a workflow from KNIX to AWS. 

First, install the required AWS SDK. Please note that you need to configure your credentials for using this SDK, e.g by adding your credentials to ~/.aws/config:


In [69]:
pip install boto3

  Using cached https://files.pythonhosted.org/packages/8f/47/fd52106b41769acac53dfe1923bb363d6e4cc583a3a951c54a85a415593f/boto3-1.14.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/07/cb/5f001272b6faeb23c1c9e0acc04d48eaaf5c862c17709d20e3469c6e0139/jmespath-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/e5/df302e8017440f111c11cc41a6b432838672f5a70aa29227bf58149dc72f/urllib3-1.25.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/cd/a6aa959dca619918ccb55023b4cb151949c64d4d5d55b3f4ffd7eee0c6e8/docutils-0.15.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/p

Now import the required librares:

In [70]:
import json
from zipfile import ZipFile
from mfn_sdk import MfnClient
import boto3
from botocore.exceptions import ClientError

get a boto3 client object for the StepFunctions service:

In [71]:
client_sf = boto3.client('stepfunctions')

get a boto3 client object for the Identity and Access Management service:

In [72]:
# Create IAM client
iam = boto3.client('iam')

create an execution role for the AWS Stepfunctions service 

In [74]:
try: 
    response = iam.get_role(RoleName='tutorialStepFunctionsPolicy')
    print(response['Role']['Arn'])
    awsSfRoleArn  = response['Role']['Arn']
except ClientError as e:
    print(e)
    response = iam.create_role(
        Path='/service-role/',
        RoleName='tutorialStepFunctionsPolicy', 
        AssumeRolePolicyDocument='{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "states.eu-central-1.amazonaws.com"}, "Action": "sts:AssumeRole"}]}',
        Description='KNIX migration tutorial',
        MaxSessionDuration=3600)
    awsSfRoleArn  = response['Role']['Arn']

arn:aws:iam::218181671562:role/service-role/tutorialStepFunctionsPolicy


In [75]:
awsSfRoleArn = response['Role']['Arn']

Now use your credentials to get a knix client object:

In [76]:
client_mfn = MfnClient(
    #mfn_url="http://knix.io/mfn",
    mfn_url="http://localhost:8080",
    mfn_user="mfn@mfn",
    mfn_password="mfn",
    mfn_name="KS",
    proxies={"http_proxy": "None", "https_proxy": "None"}
    )

We need a few parameters to prepare the data access to AWS services suchas as Lambda and Stepfunctions:
* lambdaprefix: the ARN name prefix for AWS Lambda functions
* awsSfRoleName: the name (ARN) allowing the boto3 client to access the the AWS Stepfunctions service
* knixWfName: the name of the source workflow on KNIX
* sfWFName: the name of the target workflow on AWS Stepfunctions

In [90]:
user_id = str(boto3.resource('iam').CurrentUser().arn.split(":")[4])
lambdaPrefix = "arn:aws:lambda:eu-central-1:%s:function:" % user_id 
awsSfRoleName = awsSfRoleArn  
sfWFName = "CallCenterStateMachine"
knixWfName = "test_wf_knix" 

In [91]:
for w in client_mfn.workflows:
    if w.name == knixWfName:
       knixWf = w.json
    else:
        raise Exception("Error: knix workflow not found!")

Now, knixWf contains the source workflow definition, now lets get the corresponding KNIX Function definitions. The names of these functions need to be translated to valid ARNs for the transfer to AWS StepFunctions using a name prefix.   

In [92]:
knixWf = json.loads(knixWf)
#print("Processing %s ... " % knixWfName)

for att, val in knixWf['States'].items():
    if "Resource" in list(val.keys()):
        val['Resource'] =  lambdaPrefix + val['Resource']
        print ("processing " + val['Resource'].split(":")[-1])
    else:
        print("processing Non-Task state")        
        pass

processing AssignCaseFunction
processing CloseCaseFunction
processing EscalateCaseFunction
processing Non-Task state
processing Non-Task state
processing OpenCaseFunction
processing WorkOnCaseFunction


Let's check the resulting StepFunctions workflow definition json before uploading it to AWS:

In [93]:
sf_def = json.dumps(knixWf, indent = 6)
#print(sf_def)

Now let's upload the definition using the boto3 client object using the roleArn parameter defined previously. Generate an error  if the name already exists. 

In [94]:
try:
  client_sf.create_state_machine(
   name=sfWFName,
   roleArn=awsSfRoleName,
   definition=sf_def)

except ClientError as e:
    if e.response['Error']['Code'] == 'StateMachineAlreadyExists':
        print("Error: State Machine already exists at AWS Stepfunctions")
    else:
        print("Unexpected error: %s" % e)

Now list all state machines for this account:

In [96]:
#stateMachines = client_sf.list_state_machines()
#print("all published machines: ")
#for sm in stateMachines['stateMachines']:
#   print(sm['name']) #

Now execute the state machine we've just created ans AWS StepFunctions with a sample payload JSON string

In [103]:
response = client_sf.start_execution(
    stateMachineArn="arn:aws:states:eu-central-1:218181671562:stateMachine:CallCenterStateMachine", #% (user_id, sfWFName),
    input='{\"inputCaseID\" : \"001\"}'
)

Executions on AWS Stepfunctions are asyncronous. From the start_execution response we can get an "executionArn" which we can use to query for the execution result.

In [104]:
executionArn = response['executionArn']

Let's now have a closer look at the AWS SF exection result:

In [105]:
response = client_sf.get_execution_history(executionArn=executionArn)

In [106]:
#print(response)
for ev in response['events']:        
        if "stateExitedEventDetails" in ev:
            print(str(ev["stateExitedEventDetails"]["output"]))

{"Case": "001", "Message": "Case 001: opened..."}
{"Case": "001", "Message": "Case 001: opened...assigned..."}
{"Case": "001", "Status": 0, "Message": "Case 001: opened...assigned...unresolved..."}
{"Case": "001", "Status": 0, "Message": "Case 001: opened...assigned...unresolved..."}
{"Case": "001", "Status": 0, "Message": "Case 001: opened...assigned...unresolved...escalating."}


So, our migrated KNIX State Machine executes successfully on AWS! 

![State Machine imported from KNIX into AWS Stepfunctions](stepfunctions_graph.png)